In [1]:
pip install streamlit

In [2]:
%%writefile app.py

# Import libraries
import streamlit as st
import numpy as np
import tensorflow as tf
import requests
import os
from PIL import Image, ImageDraw, ImageFont    # PIL -Python Imaging library
from io import BytesIO


# ===== Core Model Class =====
class DogIdentification:
    def __init__(self, model_path, dog_breeds):
        self.model = tf.keras.models.load_model(model_path)
        self.dog_breeds = dog_breeds
        self.image_size = (224, 224)

    def _load_image_from_url(self, url):
        response = requests.get(url)
        response.raise_for_status()
        img = Image.open(BytesIO(response.content)).convert("RGB")
        return img

    def _load_image_from_path(self, path):
        if not os.path.isfile(path):
            raise FileNotFoundError(f"File not found: {path}")
        img = Image.open(path).convert("RGB")
        return img

    def preprocess_image(self, image_input):
        try:
            if isinstance(image_input, str) and image_input.lower().startswith(("http://", "https://")):
                img = self._load_image_from_url(image_input)
            else:
                img = self._load_image_from_path(image_input)

            img = img.resize(self.image_size)
            img_array = np.array(img)
            if img_array.shape[-1] == 4:
                img_array = img_array[..., :3]
            img_array = np.expand_dims(img_array, axis=0)
            return img_array, img
        except Exception as e:
            st.error(f"Error loading image: {e}")
            return None, None

    def predict_breed(self, image_input):
        processed_image, original_image = self.preprocess_image(image_input)
        if processed_image is None:
            return "Invalid image", 0.0, None

        predictions = self.model.predict(processed_image)
        prediction_index = np.argmax(predictions)
        dog_breed = self.dog_breeds[prediction_index]
        confidence = predictions[0][prediction_index] * 100
        return dog_breed, confidence, original_image

# ===== Dog Breeds =====
dog_breeds = ['affenpinscher', 'afghan_hound', 'african_hunting_dog', 'airedale',
    'american_staffordshire_terrier', 'appenzeller', 'australian_terrier', 'basenji',
    'basset', 'beagle', 'bedlington_terrier', 'bernese_mountain_dog', 'black-and-tan_coonhound',
    'blenheim_spaniel', 'bloodhound', 'bluetick', 'border_collie', 'border_terrier', 'borzoi',
    'boston_bull', 'bouvier_des_flandres', 'boxer', 'brabancon_griffon', 'briard',
    'brittany_spaniel', 'bull_mastiff', 'cairn', 'cardigan', 'chesapeake_bay_retriever',
    'chihuahua', 'chow', 'clumber', 'cocker_spaniel', 'collie', 'curly-coated_retriever',
    'dandie_dinmont', 'dhole', 'dingo', 'doberman', 'english_foxhound', 'english_setter',
    'english_springer', 'entlebucher', 'eskimo_dog', 'flat-coated_retriever', 'french_bulldog',
    'german_shepherd', 'german_short-haired_pointer', 'giant_schnauzer', 'golden_retriever',
    'gordon_setter', 'great_dane', 'great_pyrenees', 'greater_swiss_mountain_dog', 'groenendael',
    'ibizan_hound', 'irish_setter', 'irish_terrier', 'irish_water_spaniel', 'irish_wolfhound',
    'italian_greyhound', 'japanese_spaniel', 'keeshond', 'kelpie', 'kerry_blue_terrier',
    'komondor', 'kuvasz', 'labrador_retriever', 'lakeland_terrier', 'leonberg', 'lhasa',
    'malamute', 'malinois', 'maltese_dog', 'mexican_hairless', 'miniature_pinscher',
    'miniature_poodle', 'miniature_schnauzer', 'newfoundland', 'norfolk_terrier',
    'norwegian_elkhound', 'norwich_terrier', 'old_english_sheepdog', 'otterhound',
    'papillon', 'pekinese', 'pembroke', 'pomeranian', 'pug', 'redbone', 'rhodesian_ridgeback',
    'rottweiler', 'saint_bernard', 'saluki', 'samoyed', 'schipperke', 'scotch_terrier',
    'scottish_deerhound', 'sealyham_terrier', 'shetland_sheepdog', 'shih-tzu',
    'siberian_husky', 'silky_terrier', 'soft-coated_wheaten_terrier',
    'staffordshire_bullterrier', 'standard_poodle', 'standard_schnauzer', 'sussex_spaniel',
    'tibetan_mastiff', 'tibetan_terrier', 'toy_poodle', 'toy_terrier', 'vizsla', 'walker_hound',
    'weimaraner', 'welsh_springer_spaniel', 'west_highland_white_terrier', 'whippet',
    'wire-haired_fox_terrier', 'yorkshire_terrier']

# ===== Model =====
model_path = "20250531-10451748688334_ResNet50.keras"  # the dog-breed-identification model
predictor = DogIdentification(model_path, dog_breeds)

# ===== Streamlit App =====
st.set_page_config(page_title="Dog Breed Identifier", page_icon="🐾")
st.markdown(
    """
    <style>
        .main {
            background-color: black;
        }
        .result {
            font-size: 32px;
            font-weight: bold;
            color: #A52A2A;
        }
    </style>
    """,
    unsafe_allow_html=True
)

st.title("🐾 Dog Breed Identification")

option = st.radio("Select Image Input Method", ("Upload from Path", "Enter URL"))
image_input = None

if option == "Upload from Path":
    uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "pnga"])
    if uploaded_file:
        image_input = uploaded_file
else:
    image_url = st.text_input("Enter Image URL")
    if image_url.strip():
        image_input = image_url.strip()

if st.button("Identify Breed") and image_input:
    with st.spinner("Identifying breed..."):
        # Save uploaded file temporarily if needed
        if isinstance(image_input, BytesIO) or hasattr(image_input, 'read'):
            temp_path = "temp_image.jpg"
            image = Image.open(image_input).convert("RGB")
            image.save(temp_path)
            image_input = temp_path

        dog_breed, confidence, display_image = predictor.predict_breed(image_input)

        if display_image:
            st.image(display_image, caption="Uploaded Image", width=300)
            st.markdown(f"<div class='result'>Breed: {dog_breed.replace('_', ' ').title()}</div>", unsafe_allow_html=True)
            st.success(f"Confidence: {confidence:.2f}%")
            learn_more_url = f"https://en.wikipedia.org/wiki/{dog_breed.replace('_', ' ')}"
            st.markdown(f"[Learn more about this breed]({learn_more_url})")

            # Save result image with prediction text
            draw = ImageDraw.Draw(display_image)
            font = ImageFont.load_default()
            draw.text((10, 10), f"{dog_breed} ({confidence:.2f}%)", fill="#FFFF00", font=font)

            buffer = BytesIO()
            display_image.save(buffer, format="JPEG")
            st.download_button(
                label="📥 Download Result with prediction",
                data=buffer.getvalue(),
                file_name=f"{dog_breed}.jpg",
                mime="image/jpeg"
            )

else:
    st.info("Please provide an image path or a valid URL before clicking 'Identify Breed'.")


Overwriting app.py


In [3]:
!wget -q -O - ipv4.icanhazip.com      # generates password for your url

35.186.165.75


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501   # Use the url that says "your url is: ..."yy

⠙

⠹⠸⠼⠴
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.186.165.75:8501

⠦your url is: https://late-cats-rush.loca.lt
